### 1. Import Necessasary libraries

In [1]:
import tensorflow as tf

In [5]:
tf.__version__

'2.6.0'

### 2. Import Data using Image Augumentation

Image augmentation is a technique of **altering the existing data to create some more data** for the model training process. In other words, it is the process of **artificially expanding the available dataset** for training a deep learning model.

In [2]:
from keras.preprocessing.image import ImageDataGenerator

### 2[a] Image Preprocessing using Image Augumentation

In [3]:
train_data = ImageDataGenerator(shear_range=0.2,zoom_range=0.2,rescale=1./255,horizontal_flip=True,vertical_flip=True)
test_data  = ImageDataGenerator(rescale=1./255)

In [29]:
training_folder   = train_data.flow_from_directory(directory = 'D:\DLCVNLP\Parent',
                                                   target_size=(64, 64),
                                                   color_mode='rgb',
                                                   class_mode='categorical')
                                                  #"categorical" will be 2D one-hot encoded labels,
validation_folder = test_data.flow_from_directory(directory = 'D:\DLCVNLP\Parent',target_size= (64,64))

Found 110 images belonging to 5 classes.
Found 110 images belonging to 5 classes.


In [34]:
training_folder.class_indices

{'iyan': 0, 'john': 1, 'mary': 2, 'superman': 3, 'suresh': 4}

### 3. Model Building - CNN Architecture

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

#### 3.1 Create the Pipeline

In [30]:
classifier = Sequential()

#classifier.add(Conv2D(input_shape = (256, 256,3),filters = 32, kernel_size = (3,3),strides = 1,padding = 'same',activation = 'relu'))
#classifier.add(MaxPooling2D(pool_size = (2,2),strides = 2))

classifier.add(Conv2D(input_shape = (64, 64,3),filters = 32, kernel_size = (3,3),strides = 1,padding = 'same',activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2),strides = 2))

#classifier.add(Conv2D(filters = 64, kernel_size = (3,3),strides = 1,padding = 'same',activation = 'relu'))
#classifier.add(MaxPooling2D(pool_size = (2,2),strides = 2))

classifier.add(Flatten())

classifier.add(Dense(units = 128,activation = 'relu'))
#classifier.add(Dense(units = 50,activation = 'relu'))

classifier.add(Dense(units = 5,activation = 'softmax'))

classifier.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               4194432   
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 645       
Total params: 4,195,973
Trainable params: 4,195,973
Non-trainable params: 0
_________________________________________________________________


#### 3.2 Architecture Compilation

In [31]:
classifier.compile(optimizer='adam',loss=['categorical_crossentropy'], metrics=['accuracy'])

### 4. Model Training

In [32]:
model = classifier.fit(x=training_folder,epochs=10,validation_data = validation_folder, steps_per_epoch= 2, validation_steps = 5,verbose=1)
model

Epoch 1/10
2/2 [==============================] - 85s 79s/step - loss: 3.2174 - accuracy: 0.2391 - val_loss: 5.6106 - val_accuracy: 0.2727
Epoch 2/10
2/2 [==============================] - 28s 13s/step - loss: 4.0112 - accuracy: 0.3906
Epoch 3/10
2/2 [==============================] - 26s 12s/step - loss: 2.2266 - accuracy: 0.2656
Epoch 4/10
2/2 [==============================] - 26s 14s/step - loss: 1.6346 - accuracy: 0.2969
Epoch 5/10
2/2 [==============================] - 18s 6s/step - loss: 1.3183 - accuracy: 0.4565
Epoch 6/10
2/2 [==============================] - 27s 11s/step - loss: 1.2373 - accuracy: 0.6250
Epoch 7/10
2/2 [==============================] - 18s 6s/step - loss: 1.2703 - accuracy: 0.5217
Epoch 8/10
2/2 [==============================] - 16s 5s/step - loss: 0.9544 - accuracy: 0.7609
Epoch 9/10
2/2 [==============================] - 45s 23s/step - loss: 0.9944 - accuracy: 0.6719
Epoch 10/10
2/2 [==============================] - 54s 26s/step - loss: 1.0360 - accurac

### 5. Create a h5 file for deployement

In [33]:
classifier.save('multiclass_model.h5')

### 6. Make Predictions

In [36]:
# Part 3 - Making new predictions
from keras.models import load_model
import numpy as np
from keras.preprocessing import image

test_image = image.load_img(r'D:\DLCVNLP\Image_Test_Files\superman_test\IMG_1554.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# load model
model = load_model('multiclass_model.h5')
result = model.predict(test_image)

if result[0][0] == 1:
    prediction = 'iyan'
    print(prediction)

elif result[0][1] ==1:
    prediction = 'john'
    print(prediction)
    
elif result[0][2]==1:
    print("mary")
    
elif result[0][3]==1:
    print("superman")
    
else:
    print("suresh")

john


### 7. Model Performance

In [39]:
# loss
import matplotlib.pyplot as plt
plt.plot(model.history['accuracy'], label='train loss')
#plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

TypeError: 'History' object is not subscriptable